In [17]:
import base64

def decrypt(str_data, str_key, str_alphabet):
    str_hash = ''
    # try:
    for i in range(len(str_data)):
        try:
            str_hash += str_key[i % len(str_key)]
        except:
            continue

    out = ''

    for i in range(len(str_data)):
        try:
            if str_data[i] not in str_alphabet:
                out += str_data[i]
                continue
            alphabet_count = str_alphabet.find(str_data[i])
            hash_count = str_alphabet.find(str_hash[i])
            index_calc = (alphabet_count + len(str_alphabet) - hash_count) % len(str_alphabet)
            out += str_alphabet[index_calc]
        except:
            continue
    return base64.b64decode(out).decode()
    # except Exception as e:
    #     print(e)

In [18]:
import pefile
import re
from capstone import *
from capstone.x86 import *


alphabet_str = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '

pattern1= rb'\x68...\x00' # Push pattern
# For example
# .text:00401CA2 68 0C 7F 44 00          push    offset a0mgbqzdlkr ; "0mgbQZdLKr=="


pe = pefile.PE(r"") # Put your file path

# Initialize Capstone
md = Cs(CS_ARCH_X86, CS_MODE_32)
md.skipdata = True
md.detail = True

In [19]:
insnts = []
text_section = pe.sections[0]
text_section_addr = pe.OPTIONAL_HEADER.ImageBase + text_section.VirtualAddress

# Disassemble .text section
for insnt in md.disasm(text_section.get_data(), text_section_addr):
    insnts.append(insnt)
    # print(insnt)

for i in range(len(insnts)):
    if re.match(pattern1, insnts[i].bytes):

        insn = insnts[i]
        mov_test = insnts[i+1]
        if (insn.mnemonic == "push" and
            insn.operands[0].type == X86_OP_IMM):
            if mov_test.mnemonic == "mov" and mov_test.operands[1].type == X86_OP_IMM:
                # print(insn)
            # if insn.mnemonic == "mov":
                key_inst = insnts[i]
                # print(key_inst)
                addr_key = key_inst.operands[0].imm
                print(hex(addr_key))
                break

0x42e35c


In [20]:
# Get rc4_key from .data
rdata_data_list = []
rdata_section = pe.sections[1]
rdata_section_addr =pe.OPTIONAL_HEADER.ImageBase + rdata_section.VirtualAddress
# print(hex(rdata_section_addr))

# Calculate the offset within the section
offset_within_section = addr_key - rdata_section_addr

# Get the raw data of the section
section_data = rdata_section.get_data()

# Retrieve 25 bytes of data starting from the specified address
key_value = section_data[offset_within_section:offset_within_section + 50]
index_of_null = key_value.index(b'\x00')
clean = key_value[:index_of_null]
clean = clean.decode()
# rc4_key = rc4_key.decode()
print(clean)

0504ce46646b0dc397a3c30d6692ec75


In [21]:
# get all strings from .data section
data = rdata_section.get_data()
addr_data=[]

# Split data into strings using null-termination (00)
strings = []
current_string = b""

for byte in data:
    if byte != 0:
        current_string += bytes([byte])
        # print(current_string.decode())
        # continue
    elif current_string:
        decoded_string = current_string.decode('utf-8', errors='replace')
        if all(ord(char) < 128 for char in decoded_string):
                if re.match(r'[a-zA-Z =0-9]{4,}', decoded_string):
                    strings.append(decoded_string)
        current_string = b""

# # Print extracted ASCII strings
for string in strings:
       print(string)

bad allocation
SleepConditionVariableCS
WakeAllConditionVariable
restrict(
 new
 delete
operator
 new[]
 delete[]
operator "" 
operator co_await
operator<=>
 Type Descriptor'
 Base Class Descriptor at (
 Base Class Array'
 Class Hierarchy Descriptor'
 Complete Object Locator'
FlsAlloc
FlsFree
FlsGetValue
FlsSetValue
InitializeCriticalSectionEx
CorExitProcess
AreFileApisANSI
CompareStringEx
LCMapStringEx
LocaleNameToLCID
AppPolicyGetProcessTerminationMethod
Sunday
Monday
Tuesday
Wednesday
Thursday
Friday
Saturday
January
February
March
April
June
July
August
September
October
November
December
dddd, MMMM dd, yyyy
log10
log10
asin
acos
sqrt
sinh
cosh
tanh
atan
atan2
ceil
floor
fabs
modf
ldexp
fmod
frexp
bad array new length
invalid stoi argument
stoi argument out of range
0504ce46646b0dc397a3c30d6692ec75
006700e5a2ab05704bbb0c589b88924d
f9a925
5ff842feb4356d291236d099ab7f2f93
Ct2nQXxpHMD1AmKo
AXCTd7CgGV9i1ZXrJYlm1yK7GizfgC==
BsTXOU==
NtBYOXCcUcv2DD==
1WUe11KZGfQ5OT==
JUCBXIyONUH=
AUCr10y

In [22]:
file_list = []
domain_list = []
ip_list=[]
for string in strings:
    try:
        # print(type(string))
        decrypted_string = decrypt(string, clean,alphabet_str)
        if decrypted_string is not None:
            if decrypted_string != '':
                # print(decrypted_string)

                # Find IP addresses using the regex pattern
                ip_address = re.findall(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", decrypted_string)
                
                # Find file paths using a specific pattern
                c2_paths = re.findall(r'/[a-zA-Z0-9_-]+/[a-zA-Z0-9_-]+\.php', decrypted_string)
                
                # Find version numbers using a regular expression pattern
                version_numbers = re.findall(r'\b\d\.\d{1,2}\b', decrypted_string)     

                # Find dropped file names with .exe extension using a pattern                
                dropped_files = re.findall(r'\b\w+\.exe\b', decrypted_string)
                
                # Find domain names with specified TLDs using a pattern
                domain_names = re.findall(r"^(?:[a-zA-Z0-9-]+\.)*[a-zA-Z0-9-]+\.(?:com|net|space|ltd)$", decrypted_string) 
           
            for ip_address in ip_address:
                print("IP address:", ip_address)

            for file in dropped_files:
                file_list.append(dropped_files)
                dropped = file_list[0][0]
            
            for c2_path in c2_paths:
                    print("C2 Path:", c2_path)
            
            for version in version_numbers:
                print("Amadey version:", version)

            for domain_name in domain_names:
                domain_list.append(domain_name)
                


    except:
        pass
try:
    print("Dropped file: ",dropped)
    for i in domain_list:
        print("Domain name: ",i)
except:
    pass

IP address: 77.91.124.20
C2 Path: /games/index.php
Amadey version: 3.81
Dropped file:  oneetx.exe
